In [1]:
from dh2loop import mlp

Using TensorFlow backend.


In [2]:
from __future__ import division

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords

import pandas as pd
import numpy as np
from glove import Glove
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense

import pyproj
import geopandas as gpd
import os

import json
import h5py

from keras.models import load_model
from scipy import interpolate
from itertools import product
import gdal
import ogr

import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches

[nltk_data] Downloading package stopwords to /home/ranee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ranee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ranee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#load data
DF = mlp.litho_Dataframe('../data/test3.csv', 'dh2loop_export' )

# loading word embeddings model
model = mlp.load_geovec()

number of original litho classes: 27
number of CET_Litho classes : 21
unclassified in CET_Litho: 0
number of original comment classes: 27
number of CET_Comment classes : 21
unclassified in CET_Comment: 0


In [4]:
# getting the mean embeddings of descriptions
DF = mlp.mean_embeddings('dh2loop_export.pkl', DF['Company_Litho'], DF['CET_Litho'], model)

# subseting dataset for training classifier
X, Y, X_test, Y_test, X_train, Y_train, X_validation, Y_validation = mlp.split_stratified_dataset(DF, 0.1, 0.1)# encoding lithological classes
one_enc = OneHotEncoder()
encodes = one_enc.fit_transform(Y_train).toarray()

# MLP model generation
model = Sequential()
model.add(Dense(100, input_dim=300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(units=len(DF.code.unique()), activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# training MLP model
model.fit(X_train, encodes, epochs=30, batch_size=100, verbose=2)

# saving MLP model
model.save('mlp_prob_model.h5')

# assessment of model performance
mlp.classifier_assess(model, X_validation, Y_validation)

# save lithological prediction likelihoods dataframe
mlp.save_predictions(DF, model, X, 'YSGBpredictions')

Epoch 1/30
 - 1s - loss: 2.7354 - accuracy: 0.4782
Epoch 2/30
 - 0s - loss: 1.7531 - accuracy: 0.6222
Epoch 3/30
 - 0s - loss: 1.0095 - accuracy: 0.7173
Epoch 4/30
 - 0s - loss: 0.5749 - accuracy: 0.8785
Epoch 5/30
 - 0s - loss: 0.3598 - accuracy: 0.9155
Epoch 6/30
 - 0s - loss: 0.2441 - accuracy: 0.9472
Epoch 7/30
 - 0s - loss: 0.1788 - accuracy: 0.9604
Epoch 8/30
 - 0s - loss: 0.1367 - accuracy: 0.9657
Epoch 9/30
 - 0s - loss: 0.1021 - accuracy: 0.9828
Epoch 10/30
 - 0s - loss: 0.0807 - accuracy: 0.9841
Epoch 11/30
 - 0s - loss: 0.0610 - accuracy: 0.9894
Epoch 12/30
 - 0s - loss: 0.0446 - accuracy: 0.9987
Epoch 13/30
 - 0s - loss: 0.0337 - accuracy: 1.0000
Epoch 14/30
 - 0s - loss: 0.0241 - accuracy: 1.0000
Epoch 15/30
 - 0s - loss: 0.0175 - accuracy: 1.0000
Epoch 16/30
 - 0s - loss: 0.0133 - accuracy: 1.0000
Epoch 17/30
 - 0s - loss: 0.0098 - accuracy: 1.0000
Epoch 18/30
 - 0s - loss: 0.0077 - accuracy: 1.0000
Epoch 19/30
 - 0s - loss: 0.0059 - accuracy: 1.0000
Epoch 20/30
 - 0s - l

IndexError: index 20 is out of bounds for axis 1 with size 20

In [ ]:
resampled = resampling('YSGBpredictions.pkl', 'YSGBpredictions_resampled.pkl')
#resampled.to_pickle('surveyed2_emb_depth.pkl')

In [ ]:
# DEM and shapefile must be in the same projection that the dataframe (in this
# case all were projected using crs = epsg:37255)

model = load_model('mlp_prob_model.h5')
dataPath = 'surveyed2_emb_depth.pkl'
shapefilePath = 'Moree.shp'
demPath = 'MoreeDem.tif'
#demPath = '/home/ignacio/Documents/chapter2/DEM_NSW.tif'

subset_data = get_subset(dataPath, shapefilePath)

geo2D_data = get_2D(dataPath, shapefilePath, demPath, 100)
np.save('points_Moree_Ln.npy', geo2D_data)

recake = get_3D(geo2D_data, subset_data, 100, depthMask=175, xMask=772500, yMask=6740000)
np.save('MoreeRecakeLn.npy', recake)

In [ ]:
geo2D_path = 'points_Moree_Ln.npy'
geo3D_path = 'MoreeRecakeLn.npy'

data1 = np.load(geo2D_path)
processed3D = preprocess_3Ddata(geo3D_path)
processed2D = preprocess_2Ddata(geo2D_path)
patches = [mpatches.Patch(color=litho_colors[n], label=litho_classes[n]) for n in np.unique(processed2D[:, 3])]
facecolors = np.array([litho_colors.get(i, -1) for i in range(processed3D.min(), processed3D.max() + 1)])
facecolors = facecolors[(processed3D - processed3D.min())]
facecolors = explode(facecolors)
filled = facecolors[:, :, :, -1] != 0
z, y, x = expand_coordinates(np.indices(np.array(filled.shape) + 1))

figure = plot3D(x, y, z, data1, processed2D, filled, facecolors, 'Lin')
figure.savefig('test.png', dpi=300)

In [ ]:
embeddings_path = 'NSWpredictions.pkl'

ents, quants = get_entropies(embeddings_path, model)
figure = classification_entropies(ents, quants, cols, patches, classes)

Moree = get_subset(dataPath, shapefilePath)
uncsss = uncertainties2D(Moree, 10, model, demPath)
Confusion2D = CI_points(uncsss, True)
Entropy2D = Ent_points(uncsss, True)
Ent3D = get_3D(Entropy2D, Moree, 100,
               depthMask=175, xMask=772500, yMask=6740000)
print(np.nanmax(Ent3D), np.nanmin(Ent3D), np.nanmean(Ent3D))

CI3D = get_3D(Confusion2D, Moree, 100,
              depthMask=175, xMask=772500, yMask=6740000)
print(np.nanmax(CI3D), np.nanmin(CI3D), np.nanmean(CI3D))